<a href="https://colab.research.google.com/github/JanMeow/BFHWebDev/blob/main/0802Hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load IFC

In [2]:
!pip install lark
!pip install numpy
!pip install shapely
!pip install mathutils
!pip install ifcopenshell
# !pip install compas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.4/245.4 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mathutils: filename=mathutils-3.3.0-cp311-cp311-linux_x86_64.whl size=669908 sha256=8786d26f46223ab080b720d262d192df86a0d4c004acf99fc5deb0ca4cddb5e1
  Stored in directory: /root/.cache/pip/wheels/49/bb/9e/86bd4d162535e81c0f80c64918acf23d920ed0123430d5b4a0
Successfully built mathutils
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 22.8 MB/s eta 0:00:00


In [4]:
import ifcopenshell.util
import ifcopenshell.util.element

model0 = ifcopenshell.open("ifc0.ifc") # Load our IFC model

print(model0.schema) # What's the IFC version?

targets = model0.by_type("IfcElement")
print(len(targets))

IFC4
83


In [5]:
model1 = ifcopenshell.open("ifc1.ifc") # Load our IFC model

print(model1.schema) # What's the IFC version?

targets = model1.by_type("IfcElement")
print(len(targets))

IFC4
186


# Traverse Through the ifc

In [6]:
from collections import deque


def bfs_traverse(base_node, list_contained_elements = True, func = None):

  queue = deque([base_node])
  depth = 0
  result = []

  while len(queue) !=0 :

    current_node = queue.popleft()
    print(f"CURRENT DEPTH : {depth} [TYPE] {current_node.is_a()} [GUID] ({current_node.GlobalId}) [NAME] {current_node.Name}")


    # if hasattr(current_node, "PredefinedType"):
    #   print(f"Predefined Type: {current_node.PredefinedType}")

    if func:
      print(func(current_node))
      result.append(func(current_node))

    if hasattr(current_node, "ContainsElements") and len(current_node.ContainsElements) != 0:

      for element_rel in current_node.ContainsElements:
        print(f"Contained Elements: {len(element_rel.RelatedElements)}")
        if list_contained_elements:
          for child_element in element_rel.RelatedElements:
            queue.append(child_element)




    if hasattr(current_node, "IsDecomposedBy") and len(current_node.IsDecomposedBy) != 0:
      depth +=1
      for child_rel in current_node.IsDecomposedBy:
        print(f"Number of child: {len(child_rel.RelatedObjects)}")
        for child_obj in child_rel.RelatedObjects:
          queue.append(child_obj)


  print("Function ended, No more spatial child")
  return result

In [7]:
root = model1.by_type("IfcProject")[0]
bfs_traverse(root, True, None)

CURRENT DEPTH : 0 [TYPE] IfcProject [GUID] (3miTfNL$L71QyUe3gXAoxs) [NAME] Elementproduktion Neubau MFH Steiger Kriens
Number of child: 1
CURRENT DEPTH : 1 [TYPE] IfcSite [GUID] (1SzEa5YqbDIwySB1POkj_Z) [NAME] 
Number of child: 1
CURRENT DEPTH : 2 [TYPE] IfcBuilding [GUID] (3dxf9WLe53_u7e_bPqmRyD) [NAME] Gebäude 1
Number of child: 1
CURRENT DEPTH : 3 [TYPE] IfcBuildingStorey [GUID] (0pMiGmcY16khkKwpZo2SSm) [NAME] Geschoss 1
Contained Elements: 186
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (1AgH95SW53R8Ty1FOWUW43) [NAME] 
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (1KOT4FRQHCRgSKRCSO1b3L) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (3B5zlsgnn3yQQsKceDiDur) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (05H_FC$n5FFB5WMTBJFqIp) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (0SR1SVWzvCCOr2oCYBptB6) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (2rbMQwG4TDkQBbUCssngru) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (15Q_2ddvX0Vxz2afw3sw7j) [NAME] BALKEN
C

[]

#Get Geometry Data

In [8]:
import numpy as np
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util.shape


def get_geometry_info(entity, get_global = False):


  result = None
  if hasattr(entity, "Representation"):
    if entity.Representation != None:
      settings = ifcopenshell.geom.settings()
      shape = ifcopenshell.geom.create_shape(settings, entity)

      points  = ifcopenshell.util.shape.get_vertices(shape.geometry)
      T_matrix = ifcopenshell.util.shape.get_shape_matrix(shape)
      faceVertexIndices = list(shape.geometry.faces)

      result = [T_matrix, points, faceVertexIndices]

      if get_global:
        grouped_verts = ifcopenshell.util.shape.get_vertices(shape.geometry)
        ones = np.ones(shape = (grouped_verts.shape[0],1))
        stacked = np.hstack((grouped_verts, ones))
        global_coor = stacked@ T_matrix.T
        global_coor = np.around(global_coor[:,0:-1],2)

        result.append(global_coor)

  return result


In [9]:
column = model1.by_type("IfcColumn")[0]
print(column)

proxy = model1.by_guid("3hP3TktpfEHuFubiINX0a$")
print(proxy)
print(proxy.__dir__())


#1108=IfcColumn('0kxuZxSTf9tgSukpUMxT3f',#7,'BALKEN','',$,#1111,#1116,$,.COLUMN.)
#7365=IfcBuildingElementProxy('3hP3TktpfEHuFubiINX0a$',#7,'Hüllkörper','',$,#7366,#7371,$,.NOTDEFINED.)
['ConnectedFrom', 'ConnectedTo', 'ContainedInStructure', 'Declares', 'Decomposes', 'Description', 'FillsVoids', 'GlobalId', 'HasAssignments', 'HasAssociations', 'HasContext', 'HasCoverings', 'HasOpenings', 'HasProjections', 'InterferesElements', 'IsConnectionRealization', 'IsDeclaredBy', 'IsDecomposedBy', 'IsDefinedBy', 'IsInterferedByElements', 'IsNestedBy', 'IsTypedBy', 'Name', 'Nests', 'ObjectPlacement', 'ObjectType', 'OwnerHistory', 'PredefinedType', 'ProvidesBoundaries', 'ReferencedBy', 'ReferencedInStructures', 'Representation', 'Tag', '__annotations__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__'

In [10]:
beam = model1.by_type("IfcBeam")[0]
print(beam)
beam.Representation

#411=IfcBeam('1AgH95SW53R8Ty1FOWUW43',#7,'','',$,#414,#419,$,.BEAM.)


#419=IfcProductDefinitionShape($,$,(#420))

In [11]:
print(proxy.GlobalId)

3hP3TktpfEHuFubiINX0a$


In [12]:
get_geometry_info(proxy, True)

[array([[ 0.       ,  0.       , -1.       ,  0.225    ],
        [-1.       ,  0.       ,  0.       ,  6.5100001],
        [ 0.       ,  1.       ,  0.       ,  7.2149501],
        [ 0.       ,  0.       ,  0.       ,  1.       ]]),
 array([[ 6.0200001, -0.965    , -0.225    ],
        [ 6.0200001,  0.965    , -0.225    ],
        [ 6.0200001,  0.965    ,  0.225    ],
        [ 6.0200001, -0.965    ,  0.225    ],
        [ 0.       , -0.965    ,  0.225    ],
        [ 0.       , -0.965    , -0.225    ],
        [ 0.       ,  0.965    ,  0.225    ],
        [ 0.       ,  0.965    , -0.225    ]]),
 [3,
  0,
  1,
  3,
  1,
  2,
  0,
  3,
  4,
  0,
  4,
  5,
  5,
  4,
  6,
  5,
  6,
  7,
  2,
  6,
  4,
  2,
  4,
  3,
  2,
  1,
  7,
  2,
  7,
  6,
  7,
  1,
  0,
  7,
  0,
  5],
 array([[0.45, 0.49, 6.25],
        [0.45, 0.49, 8.18],
        [0.  , 0.49, 8.18],
        [0.  , 0.49, 6.25],
        [0.  , 6.51, 6.25],
        [0.45, 6.51, 6.25],
        [0.  , 6.51, 8.18],
        [0.45, 6.51

# Create graph

Lets beging by testing the intersecting object

In [13]:
from collections import deque

class Graph:
  def __init__(self) :
    self.node_dict = {}

  def bfs_traverse(self, start_node, max_depth):
    queue = deque([start_node])
    memory = set()
    depth = 0

    while len(queue) !=0 and depth < max_depth:
      depth +=1
      current_node = queue.popleft()
      set.add(current_node.guid)



  def __len__(self):
        return len(self.node_dict)

class Node:
  def __init__(self, name, _type, guid, geom_info, psets) :
    self.name = name
    self.geom_type = _type
    self.geom_info = geom_info
    self.guid = guid
    self.psets = psets
    self.near = []

In [14]:
column = model1.by_type("IfcColumn")[0]

In [15]:
import ifcopenshell.util.element

psets = ifcopenshell.util.element.get_psets(column)
psets

{'BaseQuantities': {'Length': 4.1065,
  'CrossSectionArea': 0.0002,
  'OuterSurfaceArea': 3.7097,
  'NetSurfaceArea': 3.7097,
  'GrossVolume': 0.1971,
  'NetVolume': 0.1971,
  'GrossWeight': 0.201,
  'NetWeight': 0.201,
  'id': 1132},
 'Cadwork3dProperties': {'Group': 'AW_1.OG_04',
  'SubGroup': 'AW_1.OG',
  'Prod_Nb': '102',
  'BTA TYP': 'Tragwerk',
  'id': 1142},
 'BIMWood_Common': {'Local coordinate system': {'id': 1149,
   'type': 'IfcComplexProperty',
   'UsageName': 'Local coordinate system',
   'properties': {'Location': {'id': 1150,
     'type': 'IfcComplexProperty',
     'UsageName': 'Location',
     'properties': {'X': 5201.0000008, 'Y': -3762.9999104, 'Z': 67.0000389}},
    'Axis': {'id': 1154,
     'type': 'IfcComplexProperty',
     'UsageName': 'Axis',
     'properties': {'X': 0.0, 'Y': 0.0, 'Z': 1.0}},
    'RefDirection': {'id': 1155,
     'type': 'IfcComplexProperty',
     'UsageName': 'Reference Direction',
     'properties': {'X': 0.0, 'Y': -1.0, 'Z': 0.0}}}},
  'id': 

In [16]:

def write_to_node(current_node):
  if current_node != None:
    geom_infos = get_geometry_info(current_node, get_global = True)

    if geom_infos != None:
      psets = ifcopenshell.util.element.get_psets(current_node)
      node = Node(current_node.Name, current_node.is_a(), current_node.GlobalId, geom_infos[-1], psets)

      return node

root = model1.by_type("IfcProject")[0]
graph = Graph()


for node in bfs_traverse(root, True,write_to_node):
  if node!= None:
    graph.node_dict[node.guid] = node

CURRENT DEPTH : 0 [TYPE] IfcProject [GUID] (3miTfNL$L71QyUe3gXAoxs) [NAME] Elementproduktion Neubau MFH Steiger Kriens
None
Number of child: 1
CURRENT DEPTH : 1 [TYPE] IfcSite [GUID] (1SzEa5YqbDIwySB1POkj_Z) [NAME] 
None
Number of child: 1
CURRENT DEPTH : 2 [TYPE] IfcBuilding [GUID] (3dxf9WLe53_u7e_bPqmRyD) [NAME] Gebäude 1
None
Number of child: 1
CURRENT DEPTH : 3 [TYPE] IfcBuildingStorey [GUID] (0pMiGmcY16khkKwpZo2SSm) [NAME] Geschoss 1
None
Contained Elements: 186
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (1AgH95SW53R8Ty1FOWUW43) [NAME] 
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (1KOT4FRQHCRgSKRCSO1b3L) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (3B5zlsgnn3yQQsKceDiDur) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (05H_FC$n5FFB5WMTBJFqIp) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (0SR1SVWzvCCOr2oCYBptB6) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (2rbMQwG4TDkQBbUCssngru) [NAME] BALKEN
CURRENT DEPTH : 3 [TYPE] IfcBeam [GUID] (15Q_2ddvX0Vxz2afw3s

## Test for intersecton

In [17]:
len(graph)

186

In [18]:
proxy1 = model1.by_guid("3hP3TktpfEHuFubiINX0a$")
proxy2 = model1.by_guid("3o14as6$P1GQOeedYgIq1J")
a1 = get_geometry_info(proxy1, True)[-1]
a2 = np.around(get_geometry_info(proxy2, True)[-1])

In [19]:
a1

array([[0.45, 0.49, 6.25],
       [0.45, 0.49, 8.18],
       [0.  , 0.49, 8.18],
       [0.  , 0.49, 6.25],
       [0.  , 6.51, 6.25],
       [0.45, 6.51, 6.25],
       [0.  , 6.51, 8.18],
       [0.45, 6.51, 8.18]])

In [20]:
a2[:15]

array([[0., 0., 5.],
       [0., 0., 5.],
       [0., 0., 4.],
       [0., 0., 4.],
       [0., 7., 5.],
       [0., 7., 5.],
       [0., 7., 4.],
       [0., 7., 4.],
       [0., 7., 6.],
       [0., 7., 6.],
       [0., 7., 8.],
       [0., 7., 8.],
       [0., 0., 6.],
       [0., 0., 6.],
       [0., 0., 8.]])

In [21]:
# Get intersecting subarray

def get_intersection(arr1, arr2):
    arr1_view = arr1.view([('', arr1.dtype)] * arr1.shape[1])  # Convert rows to structured dtype
    arr2_view = arr2.view([('', arr2.dtype)] * arr2.shape[1])  # Convert rows to structured dtype
    intersection = np.intersect1d(arr1_view, arr2_view)  # Find intersecting rows
    return intersection.view(arr1.dtype).reshape(-1, arr1.shape[1])


get_intersection(a1,a2)

array([], shape=(0, 3), dtype=float64)

In [22]:
proxy1 = model1.by_guid("3hP3TktpfEHuFubiINX0a$")
proxy2 = model1.by_guid("3o14as6$P1GQOeedYgIq1J")
print(proxy1)

print(proxy2)

#7365=IfcBuildingElementProxy('3hP3TktpfEHuFubiINX0a$',#7,'Hüllkörper','',$,#7366,#7371,$,.NOTDEFINED.)
#638=IfcBuildingElementProxy('3o14as6$P1GQOeedYgIq1J',#7,'Hüllkörper','',$,#641,#646,$,.NOTDEFINED.)


In [23]:
import ifcopenshell
import multiprocessing

proxy1 = model1.by_guid("3hP3TktpfEHuFubiINX0a$")
proxy2 = model1.by_guid("3o14as6$P1GQOeedYgIq1J")

def get_adjacent_nodes(model, node, tolerance = 0.001):
  # Query neighboruing elements using BVH
  # setup BVH tree
  tree_settings = ifcopenshell.geom.settings()
  iterator = ifcopenshell.geom.iterator(
      tree_settings, model
  )

  assert iterator.initialize()
  t = ifcopenshell.geom.tree()

  while True:
      t.add_element(iterator.get_native())
      # shape = iterator.get()
      if not iterator.next():
          break

  result = t.select(proxy1,  extend=tolerance)

  adj_nodes = [connection.GlobalId for connection in result]

  return adj_nodes


print(get_adjacent_nodes(model1, proxy1))

['06nvdZ9eD2hRHyN$osqr9$', '3JLg7D0o1AxAApPtQkhzFE', '3hP3TktpfEHuFubiINX0a$', '3B5zlsgnn3yQQsKceDiDur', '1NgYdGC$53BAfmfU$o8h4$', '3o14as6$P1GQOeedYgIq1J']


In [31]:
adj_nodes = get_adjacent_nodes(model1, proxy1)
adj_nodes = [model1.by_guid(guid) for guid in adj_nodes]

We have a function get the adjacent node, now lets build the graph !

In [24]:
for key, node in graph.node_dict.items():
  adj_nodes_guid = get_adjacent_nodes(model1, node)
  node.near = [graph.node_dict[guid] for guid in adj_nodes_guid]

In [25]:
# Check if its correct
first_node = graph.node_dict["1AgH95SW53R8Ty1FOWUW43"]
print([node.guid for node in first_node.near])

['06nvdZ9eD2hRHyN$osqr9$', '3JLg7D0o1AxAApPtQkhzFE', '3hP3TktpfEHuFubiINX0a$', '3B5zlsgnn3yQQsKceDiDur', '1NgYdGC$53BAfmfU$o8h4$', '3o14as6$P1GQOeedYgIq1J']


In [26]:
def get_direct_connection(entity, graph):
  search_key = entity.GlobalId

  node = graph.node_dict[search_key]
  string1 = node.psets["Cadwork3dProperties"]["BTA TYP"]
  connections = [string1 + "//" + graph.node_dict[node.guid].psets["Cadwork3dProperties"]["BTA TYP"] for node in node.near]

  return connections

def get_indirect_connection(entity, graph):
  search_key = entity.GlobalId

  node = graph.node_dict[search_key]

In [30]:
entity = model1.by_guid("3hP3TktpfEHuFubiINX0a$")
print(get_direct_connection(entity, graph))

['Fenster 16.1//IS 15.1', 'Fenster 16.1//IW 3.2', 'Fenster 16.1//Fenster 16.1', 'Fenster 16.1//Tragwerk', 'Fenster 16.1//AW 1.1', 'Fenster 16.1//FA 2.2']


In [28]:
def dfs_traverse(base_node, max_depth = 100, depth=0, list_contained_elements = True):

  print(f"CURRENT DEPTH : {depth} [TYPE] {base_node.is_a()} [GUID] ({base_node.GlobalId}) [NAME] {base_node.Name}")

  if depth > max_depth:
    print("Max_depth is reached")
    return


  # Since the elements e.g columns and wall contained are (surprinsingly) not as part of the spatial hierachy
  if hasattr(base_node, "ContainsElements") and len(base_node.ContainsElements) != 0:
    # for element, one need to use .RelatedElement to iterate through it lol
    for element_rel in base_node.ContainsElements:
      print(f"Number of contained elements: {len(element_rel.RelatedElements)}")

      if list_contained_elements:
        for child_element in element_rel.RelatedElements:
          dfs_traverse(child_element, max_depth, depth = depth, list_contained_elements=list_contained_elements)



  # Turns out IsDecomposedBy depict more of a spatial hierachy
  for child_rel in base_node.IsDecomposedBy:
    print(f"Number of child: {len(child_rel.RelatedObjects)}")
    for child_obj in child_rel.RelatedObjects:
      dfs_traverse(child_obj, max_depth, depth = depth +1, list_contained_elements=list_contained_elements)

In [35]:
entity.GlobalId

'3hP3TktpfEHuFubiINX0a$'

In [37]:
def loop_detection(graph, start_node, depth = 0, max_depth =3):

  if depth> max_depth:
      return
  visited = set()
  routes = {}
  print(start_node.GlobalId)
  current_node = graph.node_dict[start_node.GlobalId]
  print(current_node.name)
  visited.add(current_node.guid)
  for node in current_node.near:
    if node.guid not in visited:
      loop_detection(graph, node, depth+1, max_depth)

loop_detection(graph,entity)

3hP3TktpfEHuFubiINX0a$
Hüllkörper


AttributeError: 'Node' object has no attribute 'GlobalId'